In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import os
from processingmm import batch_processing
from histopolalign import combine_images, prepare_images, semi_automatic_tool, align_folders, match_skeletons, align_imgs, get_numerical_results

# 1. Process the measurements that will be aligned

In [7]:
# get the folder in which the polarimetric measurements are stored
neoplastic_polarimetry_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'TumorMeasurements')
neoplastic_folder = '21'
polarimetry_path = os.path.join(neoplastic_polarimetry_path, neoplastic_folder)
directories = [os.path.join(neoplastic_polarimetry_path, neoplastic_folder)]

In [8]:
# process the measurements, if needed
calib_directory = os.path.join(os.getcwd().split('notebooks')[0], 'calib')
batch_processing.batch_process(directories, calib_directory, run_all = False)

Traceback (most recent call last):
  File "C:\Users\romai\Documents\processingMM\src\processingmm\batch_processing.py", line 70, in find_folder_name
    assert len(re.findall("[\d]{4}-[\d]{2}-[\d]{2}", root)) == 1
AssertionError
0it [00:00, ?it/s]


[]

# 2. Get the combined masks for the histology folders

In [9]:
# create the folde that will be used to align pathology and polarimetry
combine_images.create_to_align_folder()

# load the pathology images...
histology_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'HistologyResults')
imgs_all = combine_images.load_the_images(histology_path, Verbose = True)

# ...and process them
combine_images.get_combined_img(imgs_all, force_recompute = True, Verbose = True)

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:02<00:00, 14.16it/s]


Load the images: 2.356 seconds.
Get combined images: 0.514 seconds.


# 3. Obtain the parameters (manually using the GUI) to first align the histology and the polarimetry

### 3.1. Prepare the images

In [10]:
alignment_measurements = prepare_images.create_the_alignments(histology_path, polarimetry_path, Verbose = True)

Get polarimetry filenames and rename images: 0.005 seconds.
{'C:\\Users\\romai\\Documents\\PathologyPaper\\data\\TumorMeasurements\\21\\2022-06-29_T_HORAO-21-AF_FR_S2_1': 'C:\\Users\\romai\\Documents\\PathologyPaper\\data\\HistologyResults\\21\\HORAO-21-S2-3', 'C:\\Users\\romai\\Documents\\PathologyPaper\\data\\TumorMeasurements\\21\\2022-06-29_T_HORAO-21-BF_FR_S2_1': 'C:\\Users\\romai\\Documents\\PathologyPaper\\data\\HistologyResults\\21\\HORAO-21-S2-1'}
Create the alignment objects: 0.159 seconds.


In [11]:
alignment_measurements

[Folder path: C:\Users\romai\Documents\PathologyPaper\data\TumorMeasurements\21\2022-06-29_T_HORAO-21-AF_FR_S2_1, histology path : C:\Users\romai\Documents\PathologyPaper\data\HistologyResults\21\HORAO-21-S2-3,  status: initialized,
 Folder path: C:\Users\romai\Documents\PathologyPaper\data\TumorMeasurements\21\2022-06-29_T_HORAO-21-BF_FR_S2_1, histology path : C:\Users\romai\Documents\PathologyPaper\data\HistologyResults\21\HORAO-21-S2-1,  status: initialized]

### 3.2. Load the polarimetry, labels and histology (H&E) images for each polarimetry folder

In [6]:
alignment_measurements = prepare_images.load_and_preprocess_imgs(alignment_measurements, force_recompute = False,
                                                                        Verbose = True)


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Get the images and center of mass: 0.067 seconds.



 50%|██████████████████████████████████████████                                          | 1/2 [00:00<00:00,  1.96it/s]

Load the images: 0.442 seconds.
Get the images and center of mass: 0.118 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.82it/s]

Load the images: 0.465 seconds.


### 3.3. Semi-automatic rotation/alignement tool

In [7]:
for measurement in alignment_measurements:
    selector = semi_automatic_tool.ask_for_parameters(measurement, force_recompute = False)

# 4. Actually perform the alignment

In [8]:
align_folders.create_align_folders(alignment_measurements)


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Process the images to apply the selected changes: 0.498 seconds.
Save the processed images: 0.139 seconds.



 50%|██████████████████████████████████████████                                          | 1/2 [00:00<00:00,  1.29it/s]

Process the images to apply the selected changes: 0.485 seconds.
Save the processed images: 0.090 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.36it/s]


### 4.1. Automatic part

In [9]:
border_parameter = 5
nsamples = 400
max_distance = 150

for measurement in tqdm(alignment_measurements):
    match_skeletons.match_skeletons(measurement, border_parameter, nsamples = nsamples, max_distance = max_distance,
                                                                   Verbose = True)

 50%|██████████████████████████████████████████                                          | 1/2 [00:19<00:19, 19.94s/it]

Match the two skeletons: 19.943 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:38<00:00, 19.18s/it]

Match the two skeletons: 18.420 seconds.


### 4.2. Semi-automatic part

In [10]:
align_folders.semi_automatic_processing(alignment_measurements)

In [11]:
alignment_measurements = align_imgs.align_img_master(alignment_measurements)

### 4.3. Get the final masks and overlay them on the polarimetric parameter maps

In [12]:
align_imgs.generate_final_masks(alignment_measurements, Verbose = True)


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

create_the_image_labels_propagated: 9.510 seconds.
get_final_mask: 1.096 seconds.



 50%|██████████████████████████████████████████                                          | 1/2 [00:14<00:14, 14.13s/it]

save_all_combined_maps: 2.544 seconds.
create_the_image_labels_propagated: 16.179 seconds.
get_final_mask: 1.031 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:34<00:00, 17.42s/it]

save_all_combined_maps: 2.554 seconds.


### 4.4. Finalize and save the numerical results

In [13]:
values = get_numerical_results.create_numerical_values(alignment_measurements)

{(153, 77, 0): 1, (0, 0, 0): 7, (153, 153, 0): 2, (255, 255, 255): 0}
{(153, 77, 0): 1, (0, 0, 0): 7, (153, 153, 0): 2, (255, 255, 255): 0}
